In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
#If the program comes across any errors while training the model, please check the keras and tensorflow version installed
#Install keras version 2.3.1 and tensorflow version 2.2 for optimal performance

In [ ]:
#pip uninstall keras

Uninstalling Keras-2.4.3:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/Keras-2.4.3.dist-info/*
    /usr/local/lib/python3.6/dist-packages/docs/*
    /usr/local/lib/python3.6/dist-packages/keras/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.6/dist-packages/docs/md_autogen.py
    /usr/local/lib/python3.6/dist-packages/docs/update_docs.py
Proceed (y/n)? y
  Successfully uninstalled Keras-2.4.3


In [ ]:
#pip install keras==2.3.1

     |████████████████████████████████| 378kB 5.5MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 


In [ ]:
#pip uninstall tensorflow

Uninstalling tensorflow-2.3.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.3.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.3.0


In [ ]:
#pip install tensorflow==2.2

     |████████████████████████████████| 516.2MB 32kB/s 
     |████████████████████████████████| 3.0MB 31kB/s 
     |████████████████████████████████| 460kB 54.8MB/s 
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0


In [ ]:
zip_path_train = '/content/drive/My Drive/Python/SUBMISSION FOLDER/Flickr_8k.zip'
!cp '{zip_path_train}' .
!unzip -q Flickr_8k.zip
!rm Flickr_8k.zip

In [ ]:
import string
import numpy as np
from PIL import Image
import os
from pickle import dump, load
import numpy as np

from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers.merge import add
from keras.models import Model, load_model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout

from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [ ]:
#Data Preparation
#This is the first step for the creation of the Image Captioning model
#This process involves cleaning the descriptions and creating a word base for the
# model to use while forming sentences.

def read_file(filename):
    doc = open(filename, 'r')     
    text = doc.read()
    doc.close()
    return text


def read_descriptions(filename):
    doc = read_file(filename)  
    unclean_cap = doc.split('\n') 
    descriptions ={}
    for cap in unclean_cap[:-1]:
        img, cap = cap.split('\t')
        if img[:-2] not in descriptions:
            descriptions[img[:-2]] = [ cap ]
        else:
            descriptions[img[:-2]].append(cap)
    return descriptions



def process_captions(captions):
    table = str.maketrans('','',string.punctuation)
    for photo,sent in captions.items():   
        for i,pht_desc in enumerate(sent): 
            pht_desc.replace("-"," ")
            desc = pht_desc.split()
            desc = [word.lower() for word in desc]
            desc = [word.translate(table) for word in desc]
            desc = [word for word in desc if(len(word)>1)]
            desc = [word for word in desc if(word.isalpha())]
            pht_desc = ' '.join(desc)
            captions[photo][i]= pht_desc
    return captions

def create_vocab(descriptions):
    x = set()   
    for y in descriptions.keys():   
        [x.update(d.split()) for d in descriptions[y]]
    return x

 
def convert_to_txt(descriptions, filename):
    l = list()    
    for key, desc_list in descriptions.items():
        for desc in desc_list:
            l.append(key + '\t' + desc )
    words = "\n".join(l)    
    doc = open(filename,"w") 
    doc.write(words)
    doc.close()


flickr8k_text_path = "/content/Flickr_8k/Flickr8k_Text"   #Change Path to Flickr8k_text while executing
flickr8k_images_path = "/content/Flickr_8k/Flickr8k_Dataset"    #Change Path to Flickr8k_dataset while executing
document = flickr8k_text_path + "/" + "Flickr8k.token.txt"  


unclean_descriptions = read_descriptions(document)  
print("The size of descriptions that the project will use for learning is:",len(unclean_descriptions))
clean_descriptions = process_captions(unclean_descriptions)
vocabulary = create_vocab(clean_descriptions)
print("The size of vocabulary that the project will use for learning is:", len(vocabulary)) 
convert_to_txt(clean_descriptions, "descriptions_new.txt")

The size of descriptions that the project will use for learning is: 8092
The size of vocabulary that the project will use for learning is: 8763


In [ ]:
#Creating Xception model instance and preprocessing image before feeding to model

def extract_features(directory):
        model_xcept = Xception( include_top=False, pooling='avg' )  #model->model_xcept
        f = {}    #feature->f
        for img in tqdm(os.listdir(directory)):
            path = directory + "/" + img    #filename->path
            photo = Image.open(path)  #image->photo
            photo = photo.resize((299,299))
            photo = np.expand_dims(photo, axis=0)
            photo = photo/127.5
            photo = photo - 1.0
            feature_extracted = model_xcept.predict(photo)
            f[img] = feature_extracted
        return f

In [ ]:
#Creating feature vector and saving it as pickle file for future use

xcept_features = extract_features(flickr8k_images_path) 
dump(xcept_features, open("features_new.p","wb")) 

83689472/83683744 [==============================] - 1s 0us/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


In [ ]:
#Uncomment the below code if needed to lead the pickle file manually

#features=load(open("/content/drive/My Drive/Python/SUBMISSION FOLDER/features.p","rb"))

In [ ]:
#Reading every image and adding START and END token for RNN model to understand captions better


def read_image(filename):
    doc = read_file(filename)  
    images = doc.split("\n")[:-1] 
    return images

def read_clean_descriptions(filename, photos): 
    doc = read_file(filename)   
    descriptions = {}
    for l in doc.split("\n"):   
        words = l.split()
        if len(words)<1 :
            continue
        image, image_caption = words[0], words[1:]
        if image in photos:
            if image not in descriptions:
                descriptions[image] = []
            desc = '<start> ' + " ".join(image_caption) + ' <end>'
            descriptions[image].append(desc)
    return descriptions

def read_features(photos):
    complete = load(open("features.p","rb")) 
    x = {k:complete[k] for k in photos} 
    return x

filename = flickr8k_text_path + "/" + "Flickr_8k.trainImages.txt"   
train_imgs = read_image(filename)
train_descriptions = read_clean_descriptions("descriptions_new.txt", train_imgs)
train_features = read_features(train_imgs)

In [ ]:
#Tokeninsing each description (one-hot encoding) and saving them into pickle file


def convert_desc(descriptions):
    x = []   #all_desc->x
    for y in descriptions.keys():   
        [x.append(d) for d in descriptions[y]]
    return x

from keras.preprocessing.text import Tokenizer
def word_to_token(descriptions):
    desc_list = convert_desc(descriptions)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(desc_list)
    return tokenizer

tokenizer = word_to_token(train_descriptions)
dump(tokenizer, open('tokenizer.p', 'wb'))
vocab_size = len(tokenizer.word_index) + 1
vocab_size

7577

In [ ]:
#calculating maximum length of descriptions


def calc_size(descriptions):
    x = convert_desc(descriptions)  
    return max(len(d.split()) for d in x)
    
max_length = calc_size(unclean_descriptions)
max_length

32

In [ ]:
#create input-output sequence using image generator to provide model with multiple inputs batch wise


def data_generator(descriptions, features, tokenizer, max_length):
    while 1:
        for key, description_list in descriptions.items():
            feature = features[key][0]
            input_image, input_sequence, output_word = create_sequences(tokenizer, max_length, description_list, feature)
            yield [[input_image, input_sequence], output_word]

def create_sequences(tokenizer, max_length, desc_list, feature):
    X1, X2, y = list(), list(), list()
    for desc in desc_list:
        seq = tokenizer.texts_to_sequences([desc])[0]
        for i in range(1, len(seq)):
            in_seq, out_seq = seq[:i], seq[i]
            in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
            X1.append(feature)
            X2.append(in_seq)
            y.append(out_seq)
    return np.array(X1), np.array(X2), np.array(y)

[a,b],c = next(data_generator(train_descriptions, features, tokenizer, max_length))
a.shape, b.shape, c.shape

((47, 2048), (47, 32), (47, 7577))

In [ ]:
#Creating Model structure for merged model. The model makes use of a CNN and LSTM(RNN) for reading features
#and generating captions


from keras.utils import plot_model
def create_CNN_RNN_model(vocab_size, max_length):
    ip1 = Input(shape=(2048,))  
    xcep_layer1 = Dropout(0.5)(ip1) 
    xcep_layer2 = Dense(256, activation='relu')(xcep_layer1)  
    
    ip2 = Input(shape=(max_length,))  
    rnn1 = Embedding(vocab_size, 256, mask_zero=True)(ip2)  
    rnn2 = Dropout(0.5)(rnn1) 
    rnn3 = LSTM(256)(rnn2)  
    
    op1 = add([xcep_layer2, rnn3])  
    op2 = Dense(256, activation='relu')(op1)
    outputs = Dense(vocab_size, activation='softmax')(op2)
    
    model_combine = Model(inputs=[ip1, ip2], outputs=outputs)
    model_combine.compile(loss='categorical_crossentropy', optimizer='adam')
    
    print(model_combine.summary())
    plot_model(model_combine, to_file='model.png', show_shapes=True)
    return model_combine

In [ ]:
#Final display of all variables to make sure everything is good 


print('Size of the dataset used for training: ', len(train_imgs))
print('Number of clean description for RNN:', len(train_descriptions))
print('Number of images for CNN to extract features:', len(train_features))
print('Size of vocabulary for RNN to form captions:', vocab_size)
print('Length of longest description:', max_length)
model_final = create_CNN_RNN_model(vocab_size, max_length)
epochs = 10
steps = len(train_descriptions)


Size of the dataset used for training:  6000
Number of clean description for RNN: 6000
Number of images for CNN to extract features: 6000
Size of vocabulary for RNN to form captions: 7577
Length of longest description: 32
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32)           0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 32, 256)      1939712     input_2[0][0]                    
____________________________________________________________________

In [ ]:
#Training the merged model for multiple epoch on training data and saving each model with reduced loss 


os.mkdir("models")
for i in range(epochs):
    generator = data_generator(train_descriptions, train_features, tokenizer, max_length)
    model_final.fit_generator(generator, epochs=1, steps_per_epoch= steps, verbose=1)
    model_final.save("models/model_" + str(i) + ".h5")

In [ ]:
#print captions for 30 random images to demonstrate model capability 
import matplotlib.pyplot as plt
from matplotlib.pyplot import show

def preprocess_extract(filename, model):
        try:
            photo = Image.open(filename)    
        except:
            print("Trouble opening image file, please ensure the path provided is correct")
        photo = photo.resize((299,299))
        photo = np.array(photo)
        if photo.shape[2] == 4: 
            photo = photo[..., :3]
        photo = np.expand_dims(photo, axis=0)
        photo = photo/127.5
        photo = photo - 1.0
        feature = model.predict(photo)
        return feature


def tokenize_words(integer, tokenizer):   
  for word, index in tokenizer.word_index.items():
      if index == integer:
          return word
  return None

  
def caption_creator(model, tokenizer, photo, max_length):   
    in_text = 'start'
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        pred = model.predict([photo,sequence], verbose=0)
        pred = np.argmax(pred)
        word = tokenize_words(pred, tokenizer)    
        if word is None:
            break
        in_text += ' ' + word
        if word == 'end':
            break
    return in_text

names=[]
DATADIR = '/content/Flickr_8k/Flickr8k_Dataset'
for img in os.listdir(DATADIR):
    names.append(os.path.basename(img))

counter=0
for i in names:

  path = '/content/Flickr_8k/Flickr8k_Dataset/'+os.path.basename(i)  
  max_length = 32
  tokenizer = load(open("tokenizer.p","rb"))
  model = load_model('model_9.h5')    #Include Model path here
  model_xcept = Xception(include_top=False, pooling="avg")  
  photo = preprocess_extract(path, model_xcept)  
  image = Image.open(path)  #caption
  caption = caption_creator(model, tokenizer, photo, max_length)  
  print("\n\n")
  print(caption)  
  plt.imshow(image) 
  show()
  counter+=1
  if counter>=30:
    break